In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
#replace <your ...> with your database connection information
mode="append"
jdbc_url = "jdbc:postgresql://mypostgresdb.<>.rds.amazonaws.com:5432/<>"
config = {"user":"root", "password": "<>", "driver":"org.postgresql.Driver"}

In [ ]:
video_game_reviews = spark.read.jdbc(url=jdbc_url,table="video_game_vine_reviews", properties=config)
video_game_reviews.show(10)

In [ ]:
software_reviews = spark.read.jdbc(url=jdbc_url, table="software_vine_reviews", properties=config)
software_reviews.show(10)

In [ ]:
video_game_reviews.describe().show()

In [ ]:
software_reviews.describe().show()

In [ ]:
from pyspark.sql.functions import round
video_game_reviews_count = video_game_reviews.count()
video_game_helpful_vote = video_game_reviews.groupBy("helpful_votes").count()
video_game_helpful_vote = video_game_helpful_vote.orderBy(video_game_helpful_vote["count"].desc())
video_game_helpful_vote.withColumn('percentage', round(video_game_helpful_vote["count"]/video_game_reviews_count,4)).show()

In [ ]:
software_reviews_count = software_reviews.count()
software_helpful_vote = software_reviews.groupBy("helpful_votes").count()
software_helpful_vote = software_helpful_vote.orderBy(software_helpful_vote["count"].desc())
software_helpful_vote.withColumn('percentage', round(software_helpful_vote["count"]/software_reviews_count,4)).show()

In [ ]:
video_game_star_rating = video_game_reviews.groupBy("star_rating").count()
video_game_star_rating = video_game_star_rating.orderBy(video_game_star_rating["count"].desc())
video_game_star_rating.withColumn('percentage', round(video_game_star_rating["count"]/video_game_reviews_count,4)).show()

In [ ]:
software_star_rating = software_reviews.groupBy("star_rating").count()
software_star_rating = software_star_rating.orderBy(software_star_rating["count"].desc())
software_star_rating.withColumn('percentage', round(software_star_rating["count"]/software_reviews_count,4)).show()